# Prompt Engineering 파트 실습 3 - 1. 평가 기준 설정

앞서 배운 Prompt Development Cycle에서 가장 중요한 평가 기준부터 알아 볼 예정

## 평가 기준
- LLM이 어떠한 태스크를 어느 정도의 품질인지 정량적으로 또는 정성적으로 확인 할 수 있는 객관적인 지표


### 왜 평가 기준이 필요할까?
- A가 B보다 더 좋은 것 같다 --> 아닐 수 있음
  - 사람마다 기준이 다를 수 있고 기준이 명확하지 않을 수 있음
  - 또한 실제로 많은 데이터 테스팅 시 아닐 수도 있음

### LLM 평가 지표는 존재

1. MMLU (Massive Multitask Language Understanding) - github, paper
  - 여러 분야 테스트하는 객관식 시험
  - 참고로 MMLU (5 shot)의 경우 5개의 질문/정답 쌍이 Prompt로 주어졌다는 뜻
  - 깃헙: https://github.com/hendrycks/test
  - 논문: https://arxiv.org/pdf/2009.03300
2. ARC (Abstraction and Reasoning Corpus) - github, paper
  - 2차원 pixels grid 주고 특정 문제 해결 ex. 패턴 주고 일부 비워두고 어떤 색깔로 칠 할지 맞추는 문제
  - 깃헙: https://github.com/fchollet/ARC
  - 논문: https://arxiv.org/abs/1911.01547
3. HellaSwag - website, paper
  - 문장들 주고 이어지는 마지막 문장들로 가장 적합한 문장들 4개 중 하나 고르는 문제
  - 웹사이트: https://rowanzellers.com/hellaswag/
  - 논문: https://arxiv.org/pdf/1905.07830
4. TruthfulQA - github, agit
  - 할루시네이션 측정용 데이터셋이고 주어진 문제에 대한 Accuracy 측정 (문제 유형은 객관식 MC 외에 더 있음)
  - 깃헙: https://github.com/sylinrl/TruthfulQA
  - 논문: https://arxiv.org/abs/2109.07958


위 지표들 외에도 수학, 코딩 능력 등을 확인하는 지표들도 존재함

### 태스크에 적합한 평가 지표 설정이 필요한 이유
- 실사용 입장에선 사실 위 지표들간의 미미한 우위는 의미 없고 특정 태스크 점수가 중요
  - 예를 들어, 최근에 나온 Mistral 모델이 라마2 모델보다 지표 상으로는 더 우월함
  - 막상 한글 Q&A 태스크에서는 반대인 상황이라 실서비스 용도로는 후자의 성능이 더 좋은 케이스가 좀 더 많은 편
- 즉, 위 지표가 더 높다고 특정 태스크에서 무조건 더 높은건 아님
  - **태스크에 적합한 평가 지표 설정이 필요한 이유!**
  - 범용 모델을 평가해야 하다보니 대표적인 지표들을 선정한 것

### 평가 기준 예시
- 예시 1번. 인문학부터 STEM 분야까지 총 45개 주제를 포괄하는 35,030개의 전문가 수준 다지선형 선택 문제 (KMMLU)
  - 평가 기준 = 정확도
  - 매우 명확함
- 예시2. 요약
  - 요약을 잘 한다는 것을 어떻게 표현 할 수 있을까?
  - 정답 요약이 있을 때 정답과 최대한 비슷하게 요약하는 것이 될 수 있음
    - 이런 경우 Cosine Similarity나 Recall 기반 지표들 존재
  - 정답 요약이 없다면? 정답 요약의 품질이 좋지 않다면? 단순하게 "더 비슷한게" 정말 더 좋은 요약일까?

### 태스크에 적합한 평가 기법 분류

1. 사람이 평가하는 방법
2. LLM 모델이 평가하는 방법
3. 코드로 평가하는 방법

### 1. Human Based Method
- 사람이 평가하는 방법
  - 전문가 블라인드 A/B 테스트 --> ELO 리더보드
  - 2가지 답변 중에 더 좋은 답변을 선택하는 방법
  - 명확한 평가 기준

#### LMSys Chatbot Arena
- https://chat.lmsys.org/
- 동일한 질문에 대해 2개 모델의 모델로 답변 출력
- 2개의 모델의 이름은 가리고, 승/패/무 투표 이후에 모델명 공개

![](https://drive.google.com/uc?export=view&id=1mu79XpBWd4zKzr3kXVF3iAAxMAvwFG-o)



#### LMSys Chatbot Arena Leaderboard
- https://chat.lmsys.org/?leaderboard
- 50만번 이상의 모델 비교 결과를 토대로 ELO 랭킹 계산

![](https://drive.google.com/uc?export=view&id=1URmsi4BNmfzXipXZcS6Ee1Yu8DrksmUz)

#### 2. Model Based Evaluation
- GPT-4 같은 Strong LLM을 통해 평가하는 방법 i.e. LLM-as-a-judge
  - 실제로 사람이 평가하는 것과 굉장히 유사하다는 논문 결과들 존재
- 평가하는 방식에는 3가지 존재 (출처: MT-Bench 논문)
  1. Pairwise Comparison
    - 질문과 답변 2개를 받아 둘 중 어떤 답변이 더 좋은 지 또는 무승부인지 답변
  2. Single Answer Grading
    - 질문과 답변이 있을 때 답변에 점수를 매기는 것
  3. Reference-Guided Grading
    - 예시 답변을 주고 점수를 매기는 것

- MT-Bench 논문
  - https://arxiv.org/abs/2306.05685
  - GPT-4로 평가 진행
- G-Eval 논문
  - https://arxiv.org/abs/2303.16634

In [4]:
!pip install openai --quiet

In [5]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [6]:
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_API_KEY
)

In [7]:
question = '하늘은 왜 하늘색인가요?'

print(question)

하늘은 왜 하늘색인가요?


In [8]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': question}],
    temperature=0.0
)

answer_a = completion.choices[0].message.content
print(answer_a)

하늘은 파란색으로 보이는데, 이는 태양 빛이 대기 중의 공기 분자들과 상호 작용하여 파란색 빛을 흡수하고 반사하기 때문입니다. 이러한 현상을 산란이라고 하는데, 파란색 빛이 다른 색상의 빛보다 더 많이 산란되기 때문에 하늘이 파란색으로 보이게 됩니다. 따라서 우리가 보는 하늘의 색은 이러한 빛의 산란 현상으로 인해 파란색으로 보이게 되는 것입니다.


In [9]:
completion = client.chat.completions.create(
    model='gpt-4-1106-preview',
    messages=[{'role': 'user', 'content': question}],
    temperature=0.0
)

answer_b = completion.choices[0].message.content
print(answer_b)

하늘의 색깔이 대체로 하늘색, 즉 푸른색으로 보이는 이유는 대기 중의 분자들에 의한 빛의 산란 현상 때문입니다. 이 현상은 '레일리 산란(Rayleigh scattering)'이라고 불리며, 대기를 통과하는 태양광이 공기 분자, 먼지, 수증기 등에 의해 산란되는 과정에서 발생합니다.

레일리 산란은 파장이 짧은 빛(예: 파란색, 보라색)이 파장이 긴 빛(예: 빨간색, 주황색)보다 더 쉽게 산란되는 성질을 가지고 있습니다. 태양광은 다양한 색의 빛으로 구성되어 있는데, 이 중 파란색 빛의 파장이 가장 짧기 때문에 대기 중의 분자들에 의해 가장 강하게 산란됩니다. 그 결과, 하늘은 우리 눈에 파란색으로 보이게 됩니다.

그러나 하늘의 색깔은 하루 중 시간과 대기의 상태에 따라 변할 수 있습니다. 예를 들어, 해가 지평선에 가까울 때(일출이나 일몰 시)는 대기를 통과하는 빛의 경로가 길어지기 때문에 파란색 빛은 대부분 산란되어 사라지고, 빨간색이나 주황색 빛이 더 강하게 산란되어 하늘이 붉게 보이게 됩니다. 또한 구름이 많거나 대기 오염이 심할 때는 다른 색깔의 빛이 산란되어 하늘의 색깔이 달라질 수 있습니다.


In [10]:
# 평가 프롬프트 출처: MT-Bench 논문 https://arxiv.org/pdf/2306.05685.pdf (A. Prompt Templates Figure 5)

prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the responses provided by two
AI assistants to the user question displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.

[User Question]
{question}

[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]

[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""

print(prompt)

[System]
Please act as an impartial judge and evaluate the quality of the responses provided by two
AI assistants to the user question displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.

[User Question]
하늘은 왜 하늘색인가요?

[The Start of Assist

In [11]:
completion = client.chat.completions.create(
    model='gpt-4-turbo-2024-04-09',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Comparing the responses from Assistant A and Assistant B, both assistants provide accurate and relevant explanations for why the sky appears blue, citing the phenomenon of light scattering in the atmosphere. However, there are notable differences in the depth and detail of their explanations.

Assistant A's response is straightforward and correctly explains the basic concept of scattering, specifically mentioning that blue light is scattered more than other colors, which is why the sky appears blue. This explanation is accurate but somewhat limited in scope.

Assistant B, on the other hand, not only explains the scattering phenomenon but also introduces the specific term "Rayleigh scattering," which adds a layer of scientific precision to the response. Assistant B further elaborates on how different conditions, such as the time of day and atmospheric conditions, can affect the color of the sky, providing examples of how the sky can appear red or orange during sunrise or sunset and how 

#### 3. Code Based Evaluation
- 우리한테 익숙한 코드/로직을 통한 평가 방법
  - Accuracy, Precision, Recall...
  - ROUGE: A Package for Automatic Evaluation of Summaries
  - BLEU: a Method for Automatic Evaluation of Machine Translation
  - Exact Match, String Match

### 장단점 비교

1. Human Based Evalution
  - 통제된 환경을 가정 했을 때 사람이 직접 평가한 방법이라 안정적이고 신뢰 할 수 있음
  - 불특정 다수의 경우 약간의 노이즈 발생 가능
  - 전문 도메인의 경우 해당 도메인 전문가가 아닌 일반인이 평가 할 경우 정확도 및 평가 속도가 낮아질 수 있음
2. Model Based Evaluation
  - 사람 평가와 어느 정도 유사한 수준의 평가를 내릴 수 있음
  - 평가를 위해 API 호출이 필요한데 평가 데이터가 굉장히 많을 경우 천만원 이상은 금방 넘어 갈 수 있음
3. Code Based Evaluation
  - 위 방법들과 인력 고용 비용, 모델 호출 비용 등이 없는 무료 평가 방법
  - 태스크에 따라 위 방법들보다 더 정확 할 수도 있고 그러지 않을 수도 있음
  - 정확도 같은 지표를 벗어나 사람한테 적합한 답변을 선택하는데 있어서는 신뢰도가 상대적으로 떨어지는 편

### 결론
- 태스크에 적합한 사람이 평가하는 방법이 가장 좋음
- 그러나 현실적으로 모델이 평가하는 방법도 충분히 사용 가능
- 비용 및 속도 이슈가 없는 코드 기반 평가는 자주 활용하는 것을 권장
- 정량적인 평가와 정성적인 평가 모두 하는게 가장 이상적인 케이스
- **실서비스로 봤을 때 최종 지표는 결국 사용자 피드백**
  - 고도화 과정을 명확하게 객관적으로 observe 하기 위해서 명확한 평가 기준을 잡는 것

### 정리
- LLM 평가하는 공식적인 방법론은 존재함
  - 그러나 이 방법론들은 범용 모델의 평가 방법론이라 태스크에 적합한 평가 기준 설정이 필요
- 평가 기준에는 총 3가지 방법이 존재함
  1. 사람이 평가하는 방법
  2. 모델이 평가하는 방법 (ex. MT-Bench, G-Eval)
  3. 코드로 평가하는 방법 (ex. Accuracy, ROUGE)